# Meanings of mutability

## Review: Objects represent ideas

Recall the distinction, in programming, between **identity** and **equality**.

### Identity

Objects are **identical** when they are really the same object: existing at the same time, and occupying the same region of memory.

Python's `is` operator checks if objects are identical. Calling `id` on an object returns an integer that represents the object's identity: an object's `id` never changes, and is never equal to the `id` of any other object that exists at the same time. (Objects whose lifetimes do not overlap can have the same `id`.) As an implementation detail of CPython, `id` returns the address of the object.

### Equality

Object are **equal** when they represent the same thing. For example, we can represent the number *one billion* with an instance of `int`. All instances of `int` that represent that number have the same value, and they are thus equal, but they may be the same or different objects.

The `==` operator checks if objects are equal. The default behavior of `==` is that objects are only equal to themselves. That is, when you write a class, if you don't provide custom equality comparison behavior by overriding `__eq__`, and you don't inherit from a class that does, then your object is equal only to itself and not equal to any other object. This is a reasonable default, because, without custom `__eq__` logic, there is no way for a Python implementation to know if two different objects represent the same thing.

#### A subtlety: We only model what we need

It is possible to write a class whose instances sometimes conceptually represent the same thing, yet are still not equal, because the class does not customize equality comparison. Such a design is questionable, but not necessarily a bug. We may know it is not useful to compare the instances for equality, or that it is a poor use of developer resources to implement equality comparison at this time, yet judge that it is acceptable for the type to go into production. If the type is a non-public implementation detail, we are likely to be justified in our beliefs about how it will be used.

In situations like these, though, what is going on is that *our objects model some important things about what they represent, but do not model those things' sameness and difference.*

## Two (major) kinds of mutability

### ① Of value

The most frequently relevant meaning of mutability in Python is "its value can change."

An object is thus *immutable*, in this sense, when its value is invariant: the object never changes in a way that affects its behavior in equality comparison.

This is the meaning of "mutable" and "immutable" that applies to the ubiquitous and correct advice that mutable objects should not be hashable, and that immutable objects should usually be hashable. Uses of the words "mutable" and "immutable" in this project, except where otherwise stated, are with this meaning of mutability.

#### A subtlety: The "cannot" in "cannot change"

Consider this class:

In [1]:
class Point:
    
    def __init__(self, x, y):
        self._x = x
        self._y = y
    
    def __repr__(self):
        return f'{type(self).__name__}({self.x!r}, {self.y!r})'
    
    def __eq__(self, other):
        if isinstance(other, type(self)):
            return self.x == other.x and self.y == other.y
        return NotImplemented
    
    def __hash__(self):
        return hash((self.x, self.y))
    
    @property
    def x(self):
        return self._x
    
    @property
    def y(self):
        return self._y

We regard instances of that class to be immutable, because their behavior in equality comparison is determined by their `x` and `y` attributes, which are read-only:

In [2]:
p = Point(10, 20)
q = Point(10, 20)
p == q

True

In [3]:
p.y = 25

AttributeError: can't set attribute 'y'

But the `Point` class can only satisfy its invariants if client code follows the rules. Here, we misuse `Point` by violating encapsulation:

In [4]:
p._y = 25

In [5]:
p == q

False

We have changed the value of an object we consider immutable. This does *not* relate to ambiguity between conceptually separate senses of immutability: we consider objects immutable when their values cannot change, yet we have changed the value of such an object.

What's going on is that, by "*cannot* change," we mean that, if the code that uses a `Point` object follows expected rules, then its value never changes. To illustrate that this notion of "cannot" is meaningful, and not too weak, consider another way to break the rules:

In [6]:
import unittest.mock

In [7]:
p = Point(10, 20)
q = Point(10, 20)
p == q

True

In [8]:
# When we patch Point.__eq__ to do something different, the Point
# class is obviously not responsible for what we choose to make it do.
with unittest.mock.patch('__main__.Point.__eq__', object.__eq__):
    print(p == q)

False


Or this way:

In [9]:
type(q)

__main__.Point

In [10]:
p == q

True

In [11]:
class NotPoint:
    pass

In [12]:
q.__class__ = NotPoint  # Do the weirdest thing in Python.
type(q)

__main__.NotPoint

In [13]:
p == q

False

#### Changes that don't affect equality comparison

Some objects are immutable in this sense--their value never changes--but changes that don't contribute to equality comparison are still very important. For example, iterators generally compare equal only to themselves, but they are exhausted by iteration. The *the whole point of them* is that each call to `next` changes them.

In [14]:
a = [10, 20, 30]
it = iter(a)
hash(it)  # Iterators should be hashable, and they are.

191032015077

In [15]:
it2 = iter(a)
it == it2  # An iterator is equal only to itself.

False

In [16]:
next(it)  # Gets the first element, and ADVANCES the iterator.

10

In [17]:
next(it)  # Different result, because next changed something!

20

The paradox is resolved by considering what an iterator represents. Each iterator object represents *a particular act of iteration*. That is, an iterator represents an ongoing process of iterating through something. That ongoing process goes through different steps; the iterator can be in different states. But it is separate from all other such ongoing processes.

But perhaps this dismisses the paradox too easily. Another way to explain it is to say that iterators' reference-based equality comparison behavior is what *makes* an iterator represent an ongoing process of iterating through something. 

For example, suppose iterators were designed so when two iterators have the same type, and are iterating over the same collection object, and are at the same point in the collection, they would be equal. (That is, roughly speaking, the expected behavior of iterators in C++.) In that case, we would probably not say that they iterators were wrongly designed. We would say they represent *a particular step in a process of iteration*.

Observe that most uses of iterators are unrelated to which of these two descriptions is true. The limitation of the notion of immutability in which iterators are immutable is not that it is wrong, ambiguous, or not useful, but that, in a given situation, it may or may not be what you care about.

### ② Of state

Another, different meaning of mutability is "the state it holds can change."

#### What is state?

State is what way something is, of all the ways it can be.

For example, if you flip two coins and can distinguish which is which, there are four possible states. If you can't distinguish, so that "heads, tails" and "tails, heads" are the same state, then there are three possible states.

A system with $n$ possible states has $\log_2 n$ bits of information.

#### How do objects hold state?

Objects usually hold state in their attributes, stored in instance dictionaries or slots. Instances of built-in sequences types like `list` and `tuple`, and built-in set types like `set` and `frozenset`, hold state in their elements. Instances of built-in mapping types like `dict` hold state in their keys and values.

Objects in Python know their type. If we are only interested in the distinctions between different instances of a type, then a singleton holds no state. For example, if a variable always refers to an instance of `NoneType`, then it is guaranteed to refer to `None`, the only instance of that type. This system contains zero bits of information.

So when we say things like "there are only two possible states for a `bool`" or "there is only one possible state for a `tuple` of length zero" we are holding some information fixed--in the first case, the type, and in he second, both the type and `len`--and talking about what varies.

***FIXME: Write the rest of this.***